In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:

from langchain import OpenAI
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import openai
from langchain.chains.question_answering import load_qa_chain
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
import textwrap


def audio2Text(fileName,api_key):

    audio_file = open(fileName,"rb")
    client = openai.OpenAI(api_key=api_key)
    transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
    )
    #print(transcript)
    return transcript




# Function to answer question
def answer_question(documents,query):
    # """
    # It performs question answering by using the document search object to find the most similar documents to the input query,
    # and then passes the documents to the question answering chain with the input query as the question.
    # The answer to the question is returned as output.
    # """

    chain = load_qa_chain(llm=OpenAI(), chain_type="map_reduce")
    #query = "what is the total number of AI publications?"
    txt = chain.run(input_documents=documents, question=query)
    return txt

# Convert Audio to Text
def processUploadedFile(uploaded_file,api_key):
    if uploaded_file is not None:

        text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
      )
    raw_text = audio2Text(uploaded_file,api_key)
    texts = text_splitter.split_text(raw_text)
    return texts

# Summarize the provided Audio
def summarize_response(llm,docs):
    # Define prompt
    prompt_template = """Write a concise summary of the following:
    "{text}"
    CONCISE SUMMARY:"""
    prompt = PromptTemplate.from_template(prompt_template)

    # Define LLM chain
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # Define StuffDocumentsChain
    stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")


    return stuff_chain.run(docs)
    # Text summarization
    #chain = load_summarize_chain(llm, chain_type='map_reduce')
    #return chain.run(docs)


# Main function
def main():



    api_key = input("Provide your OPENAI Key : ")

    os.environ["OPENAI_API_KEY"] = api_key
    #wrapper = TextWrapper()

    # Upload mp3 file

    uploaded_file = '/content/sample_data/sample1.wav'
    convertedtext = ''
    convertedtext = processUploadedFile(uploaded_file,api_key)
    print("Audio Converted to Text: "+convertedtext[0])





    embeddings = OpenAIEmbeddings()
    # Instantiate the LLM model
    llm = OpenAI(temperature=0, openai_api_key=api_key)
    # Split text

    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(convertedtext[0])
    # Create multiple documents

    docs = [Document(page_content=t) for t in texts]

    summarize_txt = summarize_response(llm,docs)
    print("Summary :" +summarize_txt)

    query = input("Query on Data:")
    txt = answer_question(docs,query)
    print('Output:', txt)



if __name__ == '__main__':
    main()





Provide your OPENAI Keysk-IXIziutND7WHAjQp7FkwT3BlbkFJMoNhfIYINTbsJiuZyP9q
Audio Converted to Text: Generative AI is a type of artificial intelligence technology that can produce various types of content, including text, imagery, audio, and synthetic data. The recent buzz around generative AI has been driven by the simplicity of new user interfaces for creating high-quality text, graphics, and videos in a matter of seconds. The technology, it should be noted, is not brand new. Generative AI was introduced in the 1960s in chatbots. But it was not until 2014, with the introduction of Generative Adversarial Networks, or GANs a type of machine learning algorithm that generative AI could create convincingly authentic images, videos, and audio of real people. On the one hand, this newfound capability has opened up opportunities that include better movie dubbing and rich educational content. It also unlocked concerns about deepfakes, digitally forged images or videos, and harmful cybersecurit